In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip install ultralytics --quiet

In [ ]:
data_yaml = """
train: /content/drive/MyDrive/new-data/dataset/train/images
val: /content/drive/MyDrive/new-data/dataset/valid/images
test: /content/drive/MyDrive/new-data/dataset/test/images

nc: 11
names: ['LED', 'LED-Board', 'LED-scratched', 'TVS', 'TVS-damaged', 'fuse-resistor', 'fuse-resistor-missing', 'led-driver-ic', 'led-driver-ic-damaged', 'mosfet', 'mosfet-damaged']

roboflow:
  workspace: junaid-so4ba
  project: led-missing-component
  version: 4
  license: MIT
  url: https://universe.roboflow.com/junaid-so4ba/led-missing-component/dataset/4
"""

with open("pcb_data.yaml", "w") as f:
    f.write(data_yaml)

print(open("pcb_data.yaml").read())

In [ ]:
from ultralytics import YOLO
import os

output_project_path = "/content/drive/MyDrive/new-data/dataset/new_dataset-training_results"
os.makedirs(output_project_path, exist_ok=True)

model = YOLO("yolo11n-seg.pt")

# Training configuration
model.train(
    data="pcb_data.yaml",
    epochs=100,
    imgsz=640,
    batch=8,
    workers=4,
    name="pcb_defect_segmentation_nano",
    freeze=10,
    project=output_project_path,
    patience=10,
    lr0=0.001,
    pretrained=True
)


In [ ]:
import torch


model = YOLO("/content/drive/MyDrive/new-data/dataset/new_dataset-training_results/pcb_defect_segmentation_nano/weights/best.pt")

required_classes = {0, 1, 3, 5, 7, 9}
faulty_classes = {
    2,  # LED-scratched
    4,  # TVS-damaged
    6,  # fuse-resistor-missing
    8,  # led-driver-ic-damaged
    10, # mosfet-damaged
}

def classify_good_faulty(image_path, model, required_classes, faulty_classes):
    results = model.predict(source=image_path, imgsz=640, conf=0.25, verbose=False)
    boxes = results[0].boxes
    detected_classes = set(boxes.cls.cpu().numpy().astype(int))
    class_names = model.names
    detected_names = {class_names[c] for c in detected_classes}

    status = ""
    reason = ""
    detected_faults = faulty_classes.intersection(detected_classes)

    if detected_faults:
        status = "FAULTY"
        fault_names = {class_names[c] for c in detected_faults}
        reason = f"Detected fault condition(s): {', '.join(fault_names)}"
    elif required_classes.issubset(detected_classes):
        status = "GOOD"
        reason = "All required components present and no faults detected."
    else:
        status = " FAULTY"
        missing_classes = required_classes - detected_classes
        missing_names = {class_names[c] for c in missing_classes}
        reason = f"Missing required component(s): {', '.join(missing_names)}"

    print(f"Detected components: {detected_names if detected_names else 'None'}")
    print(f"Classification: {status}")
    print(f"Reason: {reason}")

    results[0].show()
    return status, reason


test_image = "/content/drive/MyDrive/new-data/dataset/valid/images/IMG_20260116_104137449_HDR_jpg.rf.985f51c122a98b26b842b9a1f40df35f.jpg"
classify_good_faulty(test_image, model, required_classes,faulty_classes)

In [ ]:
import glob

test_images = glob.glob("/content/drive/MyDrive/new-data/dataset/test/images/*.jpg")

good, faulty = 0, 0
for img in test_images:
    status = classify_good_faulty(img, model, required_classes,faulty_classes)
    if "GOOD" in status:
        good += 1
    else:
        faulty += 1

print(f"\n Good PCBs: {good}")
print(f" Faulty PCBs: {faulty}")